# Exercise 3: Finite size scaling

In this exercise, we will determine the critical temperature and critical exponents of the
Ising model.

a) First, we need finite size data. Since generating the data takes some time, it is useful
to save it to disk. Generate your own finite size data from last weeks program (which
can take some time to get well converged results), and/or download finite size data
provided on the GitLab page. Inspect generate_data.py to find out how the data
is structured and how to load the data. Plot the specific heat CV and magnetic
susceptibility χ.

Recall: 
$$ C_v = \frac{1}{k_BT^2} \left(\left<E^2\right> - \left<E\right>^2\right)  $$

$$ \chi = \frac{1}{k_BT} \left(\left<M^2\right> - \left<M\right>^2\right) $$


In [ ]:
%matplotlib qt5
from ising_model import IsingModel
import numpy as np
from matplotlib import pyplot as plt


In [ ]:
def calc_Cv(E: np.ndarray, T: float, kb: float = 1) -> float:
    mean_E = np.mean(E)
    mean_E2 = np.mean(E**2)
    return 1/(kb*T**2) * (mean_E2 - mean_E**2)

def calc_chi(M: np.ndarray, kbT: float) -> float:
    mean_M = np.mean(M)
    mean_M2 = np.mean(M**2)
    return 1/(kbT) * (mean_M2 - mean_M**2)
